In [2]:
import os
import codecs
import re
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from gensim.test.utils import common_texts

from gensim.models import Word2Vec

In [3]:
stopwords = ["баш","без","биће","био","бити","близу","број","дана","данас","доћи","добар","добити","док","доле","дошао","други","дуж","два","често","чији","где","горе","хвала","и","ићи","иако","иде","има","имам","имао","испод","између","изнад","изван","изволи","један","једини","једном","јесте","још","јуче","кад","како","као","кога","која","које","који","кроз","мали","мањи","мисли","много","моћи","могу","мора","морао","на","наћи","наш","негде","него","некад","неки","немам","нешто","није","ниједан","никада","нисмо","ништа","њега","његов","њен","њих","њихов","око","около","она","онај","они","оно","осим","остали","отишао","овако","овамо","овде","ове","ово","о","питати","почетак","поједини","после","поводом","правити","пре","преко","према","први","пут","радије","сада","смети","шта","ствар","стварно","сутра","сваки","све","свим","свугде","тачно","тада","тај","такође","тамо","тим", "у", "учинио","учинити","умало","унутра","употребити","узети","ваш","већина","веома","видео","више","захвалити","зашто","због","желео","жели","знати"]

In [4]:
dataset_path = '../acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        data = data.lower() #lowercase
        data = TAG_RE.sub('', data) #remove html tags
        data = data.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
        data_split = data.split() #remove extra whitespaces
        data_split = [word for word in data_split if word not in stopwords and not word.isnumeric()]
#         data = " ".join(data_split)
        dataset[file_path] = data_split

In [191]:
for key, data in dataset.items():
    print(key, data[:10])
    break

1275.html ['\ufeff', 'zakon', 'основу', 'члана', 'став', 'тачка', 'устава', 'републике', 'србије', 'доносим']


In [5]:
corpus = list(dataset.values())
model = Word2Vec(sentences=corpus, size=300, window=5, min_count=4, workers=4)

In [209]:
model.wv.most_similar('педофилија')

300

In [6]:
def dataset_to_vectors(dataset, model):
    """Changes the values in dataset. Returns X which is just the values from the dataset"""
    for key, value in dataset.items():
        wordvecs = []
        for word in value:
            try:
                wordvecs.append(model.wv[word])
            except KeyError:
                continue
        wordvecs_mean = np.array(300*[0.])
        if (wordvecs):
            wordvecs_mean = np.array(wordvecs).sum(axis=0) / float(len(wordvecs))
        dataset[key] = wordvecs_mean
    X = list(dataset.values())
    return X

In [7]:
X = dataset_to_vectors(dataset, model)

In [8]:
def query_to_vec(query, model):
    query = query.lower() #lowercase
    query = TAG_RE.sub('', query) #remove html tags
    query = query.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
    query_split = query.split() #remove extra whitespaces
    query_split = [word for word in query_split if word not in stopwords and not word.isnumeric()]
    wordvecs = []
    for word in query_split:
        try:
            wordvecs.append(model.wv[word])
        except KeyError:
            print(f'{word} not in vocabulary')
            continue
    if wordvecs:
        wordvecs_mean = np.array(wordvecs).sum(axis=0) / float(len(wordvecs))
        return wordvecs_mean
    else:
        raise ValueError('No words from the query are present in the model vocabulary')        

In [9]:
def calculate_similarity_word2vec(X, model, query, top_k=10):
    """ Vectorizes the `query` via `query_to_vec` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = query_to_vec(query, model)
    query_vec = query_vec.reshape(1,-1)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

In [10]:
def show_similar_documents(df, cosine_similarities, similar_doc_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    for index in similar_doc_indices:
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(df[index]))
        print()
        counter += 1

In [11]:
user_question = 'порез казна камата'
search_start = time.time()
sim_vecs, cosine_similarities = calculate_similarity_word2vec(X, model, user_question)
search_time = time.time() - search_start
print("search time: {:.2f} ms".format(search_time * 1000))
print()
show_similar_documents(list(dataset.keys()), cosine_similarities, sim_vecs)

search time: 6.62 ms

Top-1, Similarity = 0.5576452016830444
body: 269.html, 

Top-2, Similarity = 0.5530537366867065
body: 263.html, 

Top-3, Similarity = 0.53718501329422
body: 285.html, 

Top-4, Similarity = 0.5369895696640015
body: 254.html, 

Top-5, Similarity = 0.5312166213989258
body: 257.html, 

Top-6, Similarity = 0.5292628407478333
body: 274.html, 

Top-7, Similarity = 0.5212404131889343
body: 266.html, 

Top-8, Similarity = 0.4995863735675812
body: 256.html, 

Top-9, Similarity = 0.4900858402252197
body: 278.html, 

Top-10, Similarity = 0.4885120093822479
body: 287.html, 



In [237]:
query = '''
 ЗАКОН

о давању контрагаранције Републике Србије Србији и Црној Гори по кредиту Међународног удружења за развој (Кредит за развој приватног и финансијског сектора)

„Службени гласник РС”, број 34 од 18. априла 2006.

Члан 1.

Република Србија преузима обавезу да као контрагарант измири обавезе Србије и Црне Горе, у износу од 68.100.000,00 специјалних права вучења, по основу Споразума о кредиту за развој – Кредит за развој приватног и финансијског сектора између Савезне Републике Југославије и Међународног удружења за развој, потписаног 4. јуна 2002. године, који је потврђен Законом о потврђивању Споразума о кредиту за развој – Кредит за развој приватног и финансијског сектора између Савезне Републике Југославије и Међународног удружења за развој („Службени лист СРЈ – Међународни уговори”, број 6/02).

Члан 2.

Контрагаранција из члана 1. овог закона даје се Србији и Црној Гори, на име обавезе из Закона о задужењу Савезне Републике Југославије код Међународног удружења за развој („Службени лист СРЈ”, број 36/02).

Члан 3.

Корисник кредита по основу Споразума о кредиту за развој из члана 1. овог закона је Република Србија (у даљем тексту: Корисник кредита).

Члан 4.

Средства за отплату кредита по основу Споразума о кредиту за развој из члана 1. овог закона, обезбедиће Корисник кредита.

Корисник кредита је дужан да средства за отплату кредита обезбеђује према плану отплате за сва повучена средства, у износу који укључује доспелу главницу, трошкове на одобрена средства, манипулативне трошкове кредита и остале трошкове.

Корисник кредита ће обезбеђивати отплату кредита у буџету Републике Србије и вршити уплату доспелих обавеза на посебан рачун Србије и Црне Горе код Народне банке Србије, са искључивом наменом отплате кредита.

Члан 5.

Овај закон ступа на снагу осмог дана од дана објављивања у „Службеном гласнику Републике Србије”.
'''